In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time
import os
os.environ['JAVA_HOME'] = 'C:\\Program Files\\Java\\jdk-17.0.12'

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Write the DataFrame to a Parquet file
parquet_file = "home_sales.parquet"
df.write.parquet(parquet_file, mode="overwrite")

In [5]:
# Show the schema to verify the structure
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_4_bed_avg = """
SELECT 
    YEAR(date) AS year_sold,
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 4
GROUP BY 
    YEAR(date)
ORDER BY 
    year_sold
"""
avg_price_4_bed = spark.sql(query_4_bed_avg)
avg_price_4_bed.show()

+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_3_bed_3_bath = """
SELECT 
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND bathrooms = 3
GROUP BY 
    date_built
ORDER BY 
    year_built
"""
avg_price_3_bed_3_bath = spark.sql(query_3_bed_3_bath)
avg_price_3_bed_3_bath.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_criteria = """
SELECT 
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
WHERE 
    bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY 
    date_built
ORDER BY 
    year_built
"""
avg_price_criteria = spark.sql(query_criteria)
avg_price_criteria.show()

+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_view_rating = """
SELECT 
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM 
    home_sales
GROUP BY 
    view
HAVING 
    AVG(price) >= 350000
ORDER BY 
    view DESC
"""
avg_price_view_rating = spark.sql(query_view_rating)
avg_price_view_rating.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.8302412033081055 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_view_rating_cached = spark.sql(query_view_rating)
avg_price_view_rating_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.3128197193145752 seconds ---


In [14]:
spark.conf.set("spark.hadoop.io.file.buffer.size", "0")
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
spark.conf.set("spark.sql.parquet.writeLegacyFormat", "true")


In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
parquet_path = "/path/to/output/parquet/home_sales.parquet"
df.write.mode("overwrite").partitionBy("date_built").parquet(parquet_path)

In [16]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet(parquet_path)

In [17]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

In [18]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query_13 = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM parquet_home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
result_13 = spark.sql(query_13)
result_13.show()

# Measure query execution time
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.2107231616973877 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [20]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' still cached? {is_cached}")

Is 'home_sales' still cached? False
